# Analyzing Census ACS data for veterans trends
Done in pandas

Alan Hovorka | The Villages Daily Sun

In [3]:
from glob import glob
import os
import pandas as pd
import numpy as np
%matplotlib inline

In [4]:
# acs_dict = {
#     '2005' : 'data/acs_1_yr_vet_status_est/vets_2005_acs.csv',
#     '2006' : 'data/acs_1_yr_vet_status_est/vets_2006_acs.csv',
#     '2007' : 'data/acs_1_yr_vet_status_est/vets_2007_acs.csv',
#     '2008' : 'data/acs_1_yr_vet_status_est/vets_2008_acs.csv',
#     '2009' : 'data/acs_1_yr_vet_status_est/vets_2009_acs.csv',
#     '2010' : 'data/acs_1_yr_vet_status_est/vets_2010_acs.csv',
#     '2011' : 'data/acs_1_yr_vet_status_est/vets_2011_acs.csv',
#     '2012' : 'data/acs_1_yr_vet_status_est/vets_2012_acs.csv',
#     '2013' : 'data/acs_1_yr_vet_status_est/vets_2013_acs.csv',
#     '2014' : 'data/acs_1_yr_vet_status_est/vets_2014_acs.csv',
#     '2015' : 'data/acs_1_yr_vet_status_est/vets_2015_acs.csv',
#     '2016' : 'data/acs_1_yr_vet_status_est/vets_2016_acs.csv'
# }
acs = glob("data/*/*.csv", recursive = True)
frame = pd.DataFrame()
year_list = range(2005,2017,1)
col_list = []
df_list = []
for d in year_list:
    print(d)
for d in acs:
    print(d)
#glob the directory with the csvs into a list
#create an empty dataframe for our eventual merged/concat dataset of census 
#estimates for 12 years


2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
data/acs_1_yr_vet_status_est/vets_2005_acs.csv
data/acs_1_yr_vet_status_est/vets_2006_acs.csv
data/acs_1_yr_vet_status_est/vets_2007_acs.csv
data/acs_1_yr_vet_status_est/vets_2008_acs.csv
data/acs_1_yr_vet_status_est/vets_2009_acs.csv
data/acs_1_yr_vet_status_est/vets_2010_acs.csv
data/acs_1_yr_vet_status_est/vets_2011_acs.csv
data/acs_1_yr_vet_status_est/vets_2012_acs.csv
data/acs_1_yr_vet_status_est/vets_2013_acs.csv
data/acs_1_yr_vet_status_est/vets_2014_acs.csv
data/acs_1_yr_vet_status_est/vets_2015_acs.csv
data/acs_1_yr_vet_status_est/vets_2016_acs.csv


In [57]:
df = pd.read_csv(acs[0], header=[0,1])
df.head()
cols = df.filter(regex='\_MOE_').columns
# print(cols)
df.drop(cols, axis=1, inplace=True)
df.head()
#df_list = [pd.read_csv(f, header=[0,1]) for f in acs.drop]
#Create a list of dataframes from csv from the globed directory list
#print(df_list)

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC02_EST_VC01,HC03_EST_VC01,HC01_EST_VC03,HC02_EST_VC03,HC03_EST_VC03,HC01_EST_VC04,...,HC03_EST_VC46,HC01_EST_VC47,HC02_EST_VC47,HC03_EST_VC47,HC01_EST_VC49,HC02_EST_VC49,HC03_EST_VC49,HC01_EST_VC50,HC02_EST_VC50,HC03_EST_VC50
,Id,Id2,Geography,Total; Estimate; Civilian population 18 years and over,Veterans; Estimate; Civilian population 18 years and over,Nonveterans; Estimate; Civilian population 18 years and over,Total; Estimate; Civilian population 18 years and over - PERIOD OF SERVICE - Gulf War veterans,Veterans; Estimate; Civilian population 18 years and over - PERIOD OF SERVICE - Gulf War veterans,Nonveterans; Estimate; Civilian population 18 years and over - PERIOD OF SERVICE - Gulf War veterans,Total; Estimate; Civilian population 18 years and over - PERIOD OF SERVICE - Vietnam era veterans,...,Nonveterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over,Total; Estimate; DISABILITY STATUS - Civilian population 18 years and over - With any disability,Veterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over - With any disability,Nonveterans; Estimate; DISABILITY STATUS - Civilian population 18 years and over - With any disability,Total; Estimate; PERCENT IMPUTED - Veteran status for the population 18 years and over,Veterans; Estimate; PERCENT IMPUTED - Veteran status for the population 18 years and over,Nonveterans; Estimate; PERCENT IMPUTED - Veteran status for the population 18 years and over,Total; Estimate; PERCENT IMPUTED - Period of service for the civilian veteran population 18 years and over,Veterans; Estimate; PERCENT IMPUTED - Period of service for the civilian veteran population 18 years and over,Nonveterans; Estimate; PERCENT IMPUTED - Period of service for the civilian veteran population 18 years and over
0,0400000US01,1,Alabama,3344721,403950,2940771,(X),20.3,(X),(X),...,2940771,22.6,30.6,21.6,2.2,(X),(X),4.7,(X),(X)
1,0400000US02,2,Alaska,446969,74482,372487,(X),30.9,(X),(X),...,372487,17.3,24.8,15.8,1.4,(X),(X),4.7,(X),(X)
2,0400000US04,4,Arizona,4238996,538880,3700116,(X),16.7,(X),(X),...,3700116,16.0,25.6,14.6,1.9,(X),(X),3.6,(X),(X)
3,0400000US05,5,Arkansas,2023819,259304,1764515,(X),19.0,(X),(X),...,1764515,24.0,34.1,22.5,1.8,(X),(X),3.5,(X),(X)
4,0400000US06,6,California,25543447,2193336,23350111,(X),16.3,(X),(X),...,23350111,14.8,25.4,13.8,2.2,(X),(X),4.4,(X),(X)


### This list of dataframes is ordered by how the directory was globbed. Index 0 = 2005 data and Index = -1 is 2016

In [6]:
# https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
#Clunky method for the same above result of putting the acs list into a df list
# for f in acs:
#     df = pd.read_csv(f, header=[0,1])
#     df_list.append(df)
# frame = pd.concat(list_)
#print(col_list)

In [7]:
# print(df_list)

TypeError: 'list' object cannot be interpreted as an integer

### Various notes from other people
if the columns have the same naming scheme you could write some code that simplifies a single table, then use a for loop to do that process each table in a for loop before combining them
 
basically have a list of filenames and years, loop through each filename/year and create a simplified version of the table and put add it to a list then call pd.concat([list of tables]) to bind all the rows together
but there are a bunch of other ways to do it. sort of depends on how consistent the data is over time

My thought is you need to loop through each filename, pick out the columns you need, add a column for the year and add it to a list. After that you can smoosh it together with `pd.concat(list)`

for year, filename in dict.iteritems(): ...

### Trim back some of these columns. We can drop the margin of error columns and those that contain percents
We want the raw numbers

#### Pandas snippets

In [9]:
# Some helpful pandas snippets 
# https://gist.github.com/bsweger/e5817488d161f37dcbd2
# Get rid of non-numeric values throughout a DataFrame:
# for col in refunds.columns.values:
#   refunds[col] = refunds[col].replace('[^0-9]+.-', '', regex=True)

In [10]:
# Concatenate two DataFrame columns into a new, single column
# (useful when dealing with composite keys, for example)
# (h/t @makmanalp for improving this one!)
# df['newcol'] = df['col1'].astype(str) + df['col2'].astype(str)

In [11]:
# Create a DataFrame from a Python dictionary
# df = pd.DataFrame(list(a_dictionary.items()), columns = ['column1', 'column2'])